# Indexing and selecting data¶

The axis labeling information in pandas objects serves many purposes:

* Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display.

* Enables automatic and explicit data alignment.

* Allows intuitive getting and setting of subsets of the data set.

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

>Note
>
>The Python and NumPy indexing operators [] and attribute operator . provide quick and easy access to pandas data structures across a wide range of use cases. This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays. However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized pandas data access methods exposed in this chapter.

>Warning
>
>Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.

See the MultiIndex / Advanced Indexing for MultiIndex and more advanced indexing documentation.

See the cookbook for some advanced strategies.

## Different choices for indexing
Object selection has had a number of user-requested additions in order to support more explicit location based indexing. pandas now supports three types of multi-axis indexing.

* .loc is primarily label based, but may also be used with a boolean array. .loc will raise KeyError when the items are not found. Allowed inputs are:

  * A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

  * A list or array of labels ['a', 'b', 'c'].

  * A slice object with labels 'a':'f' (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index! See Slicing with labels and Endpoints are inclusive.)

  * A boolean array (any NA values will be treated as False).

  * A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).

See more at Selection by Label.

* .iloc is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array. .iloc will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with Python/NumPy slice semantics). Allowed inputs are:

  * A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

  * A list or array of labels ['a', 'b', 'c'].

  * A slice object with labels 'a':'f' (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index! See Slicing with labels and Endpoints are inclusive.)

  * A boolean array (any NA values will be treated as False).

  * A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).

See more at Selection by Label.

* .loc, .iloc, and also [] indexing can accept a callable as indexer. See more at Selection By Callable.

Getting values from an object with multi-axes selection uses the following notation (using .loc as an example, but the following applies to .iloc as well). Any of the axes accessors may be the null slice :. Axes left out of the specification are assumed to be :, e.g. p.loc['a'] is equivalent to p.loc['a', :, :].

| Object Type | Indexers |
|-------------|----------|
| Series | s.loc[indexer] |
| DataFrame | df.loc[row_indexer,column_indexer] |

## Basics
As mentioned when introducing the data structures in the last section, the primary function of indexing with [] (a.k.a. __getitem__ for those familiar with implementing class behavior in Python) is selecting out lower-dimensional slices. The following table shows return type values when indexing pandas objects with []:

| Object Type | Selection | Return Value Type |
|-------------|-----------|-------------------|
| Series | series[label] | scalar value |
| DataFrame | frame[colname] | Series corresponding to colname |

Here we construct a simple time series data set to use for illustrating the indexing functionality:

In [1]:
import shutil
from distutils.dir_util import copy_tree
copy_tree('archive/','data/')

['data/baseball.xlsb',
 'data/mindex_ex.csv',
 'data/banklist.html',
 'data/baseball.csv',
 'data/iris.data',
 'data/spam.html']

In [2]:
import pandas as pd
import numpy as np
import sys

In [3]:
dates = pd.date_range('1/1/2000', periods=8)
df1 = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])
df1

,A,B,C,D
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,-2.040983,0.098607,-1.503086,0.420989
2000-01-04,0.255814,-1.303977,0.954433,-1.137952
2000-01-05,0.020985,0.476644,0.850724,-0.722163
2000-01-06,0.173625,-0.699478,0.355988,1.610516
2000-01-07,0.880987,-1.717641,0.809490,1.449279
2000-01-08,-1.805341,0.762664,-0.205993,-1.595724


>Note
>
>None of the indexing functionality is time series specific unless specifically stated.

Thus, as per above, we have the most basic indexing using []:

In [4]:
s1 = df1['A']
s1[dates[5]]

0.17362502462808632

You can pass a list of columns to [] to select columns in that order. If a column is not contained in the DataFrame, an exception will be raised. Multiple columns can also be set in this manner:

In [5]:
df2 = df1.copy()
df2

,A,B,C,D
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,-2.040983,0.098607,-1.503086,0.420989
2000-01-04,0.255814,-1.303977,0.954433,-1.137952
2000-01-05,0.020985,0.476644,0.850724,-0.722163
2000-01-06,0.173625,-0.699478,0.355988,1.610516
2000-01-07,0.880987,-1.717641,0.809490,1.449279
2000-01-08,-1.805341,0.762664,-0.205993,-1.595724


In [6]:
df2[['B', 'A']] = df1[['A', 'B']]
df2

,A,B,C,D
2000-01-01,-1.139756,-0.119524,0.517904,-0.285438
2000-01-02,0.070927,2.021892,0.800103,0.118657
2000-01-03,0.098607,-2.040983,-1.503086,0.420989
2000-01-04,-1.303977,0.255814,0.954433,-1.137952
2000-01-05,0.476644,0.020985,0.850724,-0.722163
2000-01-06,-0.699478,0.173625,0.355988,1.610516
2000-01-07,-1.717641,0.880987,0.809490,1.449279
2000-01-08,0.762664,-1.805341,-0.205993,-1.595724


You may find this useful for applying a transform (in-place) to a subset of the columns.

>Warning
>
>pandas aligns all AXES when setting Series and DataFrame from .loc, and .iloc.
>
>This will not modify df because the column alignment is before value assignment.

In [7]:
df2[['A', 'B']]

,A,B
2000-01-01,-1.139756,-0.119524
2000-01-02,0.070927,2.021892
2000-01-03,0.098607,-2.040983
2000-01-04,-1.303977,0.255814
2000-01-05,0.476644,0.020985
2000-01-06,-0.699478,0.173625
2000-01-07,-1.717641,0.880987
2000-01-08,0.762664,-1.805341


In [8]:
df2.loc[:, ['B', 'A']] = df2[['A', 'B']]
df2[['A', 'B']]

,A,B
2000-01-01,-1.139756,-0.119524
2000-01-02,0.070927,2.021892
2000-01-03,0.098607,-2.040983
2000-01-04,-1.303977,0.255814
2000-01-05,0.476644,0.020985
2000-01-06,-0.699478,0.173625
2000-01-07,-1.717641,0.880987
2000-01-08,0.762664,-1.805341


The correct way to swap column values is by using raw values:

In [9]:
df2.loc[:, ['B', 'A']] = df2[['A', 'B']].to_numpy()
df2[['A', 'B']]

,A,B
2000-01-01,-0.119524,-1.139756
2000-01-02,2.021892,0.070927
2000-01-03,-2.040983,0.098607
2000-01-04,0.255814,-1.303977
2000-01-05,0.020985,0.476644
2000-01-06,0.173625,-0.699478
2000-01-07,0.880987,-1.717641
2000-01-08,-1.805341,0.762664


## Attribute access
You may access an index on a Series or column on a DataFrame directly as an attribute:

In [10]:
s3 = pd.Series([1, 2, 3], index=list('abc'))
df3 = df1.copy()
s3.b

2

In [11]:
df3.A

2000-01-01   -0.119524
2000-01-02    2.021892
2000-01-03   -2.040983
2000-01-04    0.255814
2000-01-05    0.020985
2000-01-06    0.173625
2000-01-07    0.880987
2000-01-08   -1.805341
Freq: D, Name: A, dtype: float64

In [12]:
s3.a = 5
s3

a    5
b    2
c    3
dtype: int64

In [13]:
df3.A = list(range(len(df3.index)))  # ok if A already exists
df3

,A,B,C,D
2000-01-01,0,-1.139756,0.517904,-0.285438
2000-01-02,1,0.070927,0.800103,0.118657
2000-01-03,2,0.098607,-1.503086,0.420989
2000-01-04,3,-1.303977,0.954433,-1.137952
2000-01-05,4,0.476644,0.850724,-0.722163
2000-01-06,5,-0.699478,0.355988,1.610516
2000-01-07,6,-1.717641,0.809490,1.449279
2000-01-08,7,0.762664,-0.205993,-1.595724


In [14]:
df3['AA'] = list(range(len(df3.index)))  # use this form to create a new column
df3

,A,B,C,D,AA
2000-01-01,0,-1.139756,0.517904,-0.285438,0
2000-01-02,1,0.070927,0.800103,0.118657,1
2000-01-03,2,0.098607,-1.503086,0.420989,2
2000-01-04,3,-1.303977,0.954433,-1.137952,3
2000-01-05,4,0.476644,0.850724,-0.722163,4
2000-01-06,5,-0.699478,0.355988,1.610516,5
2000-01-07,6,-1.717641,0.809490,1.449279,6
2000-01-08,7,0.762664,-0.205993,-1.595724,7


>Warning
>
>* You can use this access only if the index element is a valid Python identifier, e.g. s.1 is not allowed. See here for an explanation of valid identifiers.
>
>* The attribute will not be available if it conflicts with an existing method name, e.g. s.min is not allowed, but s['min'] is possible.
>
>* Similarly, the attribute will not be available if it conflicts with any of the following list: index, major_axis, minor_axis, items.
>
>* In any of these cases, standard indexing will still work, e.g. s['1'], s['min'], and s['index'] will access the corresponding element or column.

If you are using the IPython environment, you may also use tab-completion to see these accessible attributes.

You can also assign a dict to a row of a DataFrame:

In [15]:
x4 = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
x4.iloc[1] = {'x': 9, 'y': 99}
x4

,x,y
0,1,3
1,9,99
2,3,5


You can use attribute access to modify an existing element of a Series or column of a DataFrame, but be careful; if you try to use attribute access to create a new column, it creates a new attribute rather than a new column. In 0.21.0 and later, this will raise a UserWarning:

In [16]:
df4 = pd.DataFrame({'one': [1., 2., 3.]})
df4.two = [4, 5, 6]
df4

<ipython-input-16-8503afd9ff10>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df4.two = [4, 5, 6]


,one
0,1.0
1,2.0
2,3.0


## Slicing ranges
The most robust and consistent way of slicing ranges along arbitrary axes is described in the Selection by Position section detailing the .iloc method. For now, we explain the semantics of slicing using the [] operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:

In [17]:
s4 = df3.B.copy()
s4[:5]

2000-01-01   -1.139756
2000-01-02    0.070927
2000-01-03    0.098607
2000-01-04   -1.303977
2000-01-05    0.476644
Freq: D, Name: B, dtype: float64

In [18]:
s4[::2]

2000-01-01   -1.139756
2000-01-03    0.098607
2000-01-05    0.476644
2000-01-07   -1.717641
Freq: 2D, Name: B, dtype: float64

In [19]:
s4[::-1]

2000-01-08    0.762664
2000-01-07   -1.717641
2000-01-06   -0.699478
2000-01-05    0.476644
2000-01-04   -1.303977
2000-01-03    0.098607
2000-01-02    0.070927
2000-01-01   -1.139756
Freq: -1D, Name: B, dtype: float64

Note that setting works as well:

In [20]:
s5 = s4.copy()
s5[:5] = 0
s5

2000-01-01    0.000000
2000-01-02    0.000000
2000-01-03    0.000000
2000-01-04    0.000000
2000-01-05    0.000000
2000-01-06   -0.699478
2000-01-07   -1.717641
2000-01-08    0.762664
Freq: D, Name: B, dtype: float64

With DataFrame, slicing inside of [] slices the rows. This is provided largely as a convenience since it is such a common operation.

In [21]:
df1[:3]

,A,B,C,D
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,-2.040983,0.098607,-1.503086,0.420989


In [22]:
df1[::-1]

,A,B,C,D
2000-01-08,-1.805341,0.762664,-0.205993,-1.595724
2000-01-07,0.880987,-1.717641,0.809490,1.449279
2000-01-06,0.173625,-0.699478,0.355988,1.610516
2000-01-05,0.020985,0.476644,0.850724,-0.722163
2000-01-04,0.255814,-1.303977,0.954433,-1.137952
2000-01-03,-2.040983,0.098607,-1.503086,0.420989
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438


## Selection by label
>Warning
>
>Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.

>Warning
>
>.loc is strict when you present slicers that are not compatible (or convertible) with the index type. For example using integers in a DatetimeIndex. These will raise a TypeError.

In [23]:
df6 = pd.DataFrame(np.random.randn(5, 4),
                   columns=list('ABCD'),
                   index=pd.date_range('20130101', periods=5))
df6

,A,B,C,D
2013-01-01,0.723851,-0.211720,1.314130,-0.439573
2013-01-02,0.250914,-0.946315,0.492595,1.612376
2013-01-03,-0.892686,1.279112,0.435207,-1.079864
2013-01-04,-1.067492,-0.048729,-1.955733,-0.365553
2013-01-05,-0.477897,-1.440080,-1.382869,0.289859


In [24]:
try:
    df6.loc[2:3]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int \n'
    msg = msg + 'FIX: use a compatible index\n'
    print(msg)    

sys.exc_info(): <class 'TypeError'> 
TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int 
FIX: use a compatible index



String likes in slicing can be convertible to the type of the index and lead to natural slicing.

In [25]:
df6.loc['20130102':'20130104']

,A,B,C,D
2013-01-02,0.250914,-0.946315,0.492595,1.612376
2013-01-03,-0.892686,1.279112,0.435207,-1.079864
2013-01-04,-1.067492,-0.048729,-1.955733,-0.365553


>Warning
>
>Changed in version 1.0.0.
>
>pandas will raise a KeyError if indexing with a list with missing labels. See list-like Using loc with missing keys in a list is Deprecated.

pandas provides a suite of methods in order to have purely label based indexing. This is a strict inclusion based protocol. Every label asked for must be in the index, or a KeyError will be raised. When slicing, both the start bound AND the stop bound are included, if present in the index. Integers are valid labels, but they refer to the label and not the position.

The .loc attribute is the primary access method. The following are valid inputs:

* A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

* A list or array of labels ['a', 'b', 'c'].

* A slice object with labels 'a':'f' (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index! See Slicing with labels.

* A boolean array.

* A callable, see Selection By Callable.

In [26]:
s6 = pd.Series(np.random.randn(6), index=list('abcdef'))
s6

a    1.854032
b    0.397577
c    0.059492
d    1.124851
e   -0.207485
f   -0.638517
dtype: float64

In [27]:
s6.loc['c':]

c    0.059492
d    1.124851
e   -0.207485
f   -0.638517
dtype: float64

In [28]:
s6.loc['b']

0.3975771124476619

Note that setting works as well:

In [29]:
s6.loc['c':] = 0
s6

a    1.854032
b    0.397577
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

With a DataFrame:

In [30]:
df7 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))
df7

,A,B,C,D
a,1.808326,-1.292690,-1.097374,-0.363154
b,-1.051349,-0.798763,-0.280958,0.077838
c,-0.993829,-1.480812,0.032088,-0.498347
d,1.153185,-0.683602,0.373852,-0.324962
e,-0.789674,1.025804,1.195855,0.450637
f,-0.780291,0.785948,1.795367,-1.004865


In [31]:
df7.loc[['a', 'b', 'd'], :]

,A,B,C,D
a,1.808326,-1.292690,-1.097374,-0.363154
b,-1.051349,-0.798763,-0.280958,0.077838
d,1.153185,-0.683602,0.373852,-0.324962


Accessing via label slices:

In [32]:
df7.loc['d':, 'A':'C']

,A,B,C
d,1.153185,-0.683602,0.373852
e,-0.789674,1.025804,1.195855
f,-0.780291,0.785948,1.795367


For getting a cross section using a label (equivalent to df.xs('a')):

In [33]:
df7.loc['a']

A    1.808326
B   -1.292690
C   -1.097374
D   -0.363154
Name: a, dtype: float64

For getting values with a boolean array:

In [34]:
df7.loc['a'] > 1

A     True
B    False
C    False
D    False
Name: a, dtype: bool

In [35]:
df7.loc[:, df7.loc['a'] > 1]

,A
a,1.808326
b,-1.051349
c,-0.993829
d,1.153185
e,-0.789674
f,-0.780291


NA values in a boolean array propagate as False:

*Changed in version 1.0.2.*

In [36]:
mask7 = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")
mask7

<BooleanArray>
[True, False, True, False, <NA>, False]
Length: 6, dtype: boolean

In [37]:
df7[mask7]

,A,B,C,D
a,1.808326,-1.292690,-1.097374,-0.363154
c,-0.993829,-1.480812,0.032088,-0.498347


For getting a value explicitly:

In [38]:
# this is also equivalent to ``df7.at['a','A']``
df7.loc['a', 'A']

1.8083260103195404

### Slicing with labels
When using .loc with slices, if both the start and the stop labels are present in the index, then elements located between the two (including them) are returned:

In [39]:
s8 = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])
s8.loc[3:5]

3    b
2    c
5    d
dtype: object

If at least one of the two is absent, but the index is sorted, and can be compared against start and stop labels, then slicing will still work as expected, by selecting labels which rank between the two:

In [40]:
s8.sort_index()

0    a
2    c
3    b
4    e
5    d
dtype: object

In [41]:
s8.sort_index().loc[1:6]

2    c
3    b
4    e
5    d
dtype: object

However, if at least one of the two is absent and the index is not sorted, an error will be raised (since doing otherwise would be computationally expensive, as well as potentially ambiguous for mixed type indexes). For instance, in the above example, s.loc[1:6] would raise KeyError.

For the rationale behind this behavior, see Endpoints are inclusive.

In [42]:
s9 = pd.Series(list('abcdef'), index=[0, 3, 2, 5, 4, 2])
s9.loc[3:5]

3    b
2    c
5    d
dtype: object

Also, if the index has duplicate labels and either the start or the stop label is dupulicated, an error will be raised. For instance, in the above example, s.loc[2:5] would raise a KeyError.

For more information about duplicate labels, see Duplicate Labels.

## Selection by position
>Warning
>
>Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.

pandas provides a suite of methods in order to get purely integer based indexing. The semantics follow closely Python and NumPy slicing. These are 0-based indexing. When slicing, the start bound is included, while the upper bound is excluded. Trying to use a non-integer, even a valid label will raise an IndexError.

The .iloc attribute is the primary access method. The following are valid inputs:

* An integer e.g. 5.

* A list or array of integers [4, 3, 0].

* A slice object with ints 1:7.

* A boolean array.

* A callable, see Selection By Callable.

In [43]:
s10 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))
s10

0   -0.721791
2    0.351704
4   -0.001504
6    0.328543
8    2.130401
dtype: float64

In [44]:
s10.iloc[:3]

0   -0.721791
2    0.351704
4   -0.001504
dtype: float64

In [45]:
s10.iloc[3]

0.3285431151686854

Note that setting works as well:

In [46]:
s10.iloc[:3] = 0
s10

0    0.000000
2    0.000000
4    0.000000
6    0.328543
8    2.130401
dtype: float64

With a DataFrame:

In [47]:
df11 = pd.DataFrame(np.random.randn(6, 4),
                   index=list(range(0, 12, 2)),
                   columns=list(range(0, 8, 2)))
df11

,0,2,4,6
0,-0.773667,1.649508,-0.021440,0.132237
2,-0.642045,-1.546381,-1.659319,-1.315683
4,2.032742,-0.932632,1.766463,-0.210459
6,-2.236883,0.745316,-0.482853,0.436610
8,-0.758500,-1.277247,-0.876784,-0.058099
10,-2.584631,0.096796,1.812591,0.687361


Select via integer slicing:

In [48]:
df11.iloc[:3]

,0,2,4,6
0,-0.773667,1.649508,-0.021440,0.132237
2,-0.642045,-1.546381,-1.659319,-1.315683
4,2.032742,-0.932632,1.766463,-0.210459


In [49]:
df11.iloc[1:5, 2:4]

,4,6
2,-1.659319,-1.315683
4,1.766463,-0.210459
6,-0.482853,0.436610
8,-0.876784,-0.058099


Select via integer list:

In [50]:
df11.iloc[[1, 3, 5], [1, 3]]

,2,6
2,-1.546381,-1.315683
6,0.745316,0.436610
10,0.096796,0.687361


In [51]:
df11.iloc[1:3, :]

,0,2,4,6
2,-0.642045,-1.546381,-1.659319,-1.315683
4,2.032742,-0.932632,1.766463,-0.210459


In [52]:
df11.iloc[:, 1:3]

,2,4
0,1.649508,-0.021440
2,-1.546381,-1.659319
4,-0.932632,1.766463
6,0.745316,-0.482853
8,-1.277247,-0.876784
10,0.096796,1.812591


In [53]:
# this is also equivalent to ``df1.iat[1,1]``
df11.iloc[1, 1]

-1.5463808134291628

For getting a cross section using an integer position (equiv to df.xs(1)):

In [54]:
df11.iloc[1]

0   -0.642045
2   -1.546381
4   -1.659319
6   -1.315683
Name: 2, dtype: float64

Out of range slice indexes are handled gracefully just as in Python/NumPy.

In [55]:
# these are allowed in Python/NumPy.
x11 = list('abcdef')
x11

['a', 'b', 'c', 'd', 'e', 'f']

In [56]:
x11[4:10]

['e', 'f']

In [57]:
x11[8:10]

[]

In [58]:
s11 = pd.Series(x11)
s11

0    a
1    b
2    c
3    d
4    e
5    f
dtype: object

In [59]:
s11.iloc[4:10]

4    e
5    f
dtype: object

In [60]:
s11.iloc[8:10]

Series([], dtype: object)

Note that using slices that go out of bounds can result in an empty axis (e.g. an empty DataFrame being returned).

In [61]:
df12 = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))
df12

,A,B
0,-0.556287,-0.672443
1,-0.941971,0.974271
2,0.287531,-0.665444
3,0.080988,-0.456751
4,-0.698668,-0.091983


In [62]:
df12.iloc[:, 2:3]

""
0
1
2
3
4


In [63]:
df12.iloc[:, 2:3]

""
0
1
2
3
4


In [64]:
df12.iloc[:, 1:3]

,B
0,-0.672443
1,0.974271
2,-0.665444
3,-0.456751
4,-0.091983


In [65]:
df12.iloc[4:6]

,A,B
4,-0.698668,-0.091983


A single indexer that is out of bounds will raise an IndexError. A list of indexers where any element is out of bounds will raise an IndexError.

In [66]:
try:
    df12.iloc[[4, 5, 6]]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'IndexError: positional indexers are out-of-bounds \n'
    msg = msg + 'FIX: use correct index range \n'
    print(msg)

sys.exc_info(): <class 'IndexError'> 
IndexError: positional indexers are out-of-bounds 
FIX: use correct index range 



In [67]:
try:
    df12.iloc[:, 4]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'IndexError: single positional indexer is out-of-bounds \n'
    msg = msg + 'FIX: use correct index range \n'
    print(msg)

sys.exc_info(): <class 'IndexError'> 
IndexError: single positional indexer is out-of-bounds 
FIX: use correct index range 



## Selection by callable
.loc, .iloc, and also [] indexing can accept a callable as indexer. The callable must be a function with one argument (the calling Series or DataFrame) that returns valid output for indexing.

In [68]:
df13 = pd.DataFrame(np.random.randn(6, 4),
                    index=list('abcdef'),
                    columns=list('ABCD'))
df13

,A,B,C,D
a,-0.307620,-0.617461,2.171615,0.455244
b,-0.379891,-0.555792,0.445579,-2.429098
c,0.397711,-0.552411,0.527463,-0.962704
d,-1.632263,1.321678,0.838703,2.725299
e,-0.315488,-0.179360,-0.233477,-0.048576
f,1.922169,-2.086020,-0.136242,0.184933


In [69]:
df13.loc[lambda dfx: dfx['A'] > 0, :]

,A,B,C,D
c,0.397711,-0.552411,0.527463,-0.962704
f,1.922169,-2.086020,-0.136242,0.184933


In [70]:
df13.loc[:, lambda dfx: ['A', 'B']]

,A,B
a,-0.307620,-0.617461
b,-0.379891,-0.555792
c,0.397711,-0.552411
d,-1.632263,1.321678
e,-0.315488,-0.179360
f,1.922169,-2.086020


In [71]:
df13.iloc[:, lambda dfx: [0, 1]]

,A,B
a,-0.307620,-0.617461
b,-0.379891,-0.555792
c,0.397711,-0.552411
d,-1.632263,1.321678
e,-0.315488,-0.179360
f,1.922169,-2.086020


In [72]:
df13[lambda dfx: dfx.columns[0]]

a   -0.307620
b   -0.379891
c    0.397711
d   -1.632263
e   -0.315488
f    1.922169
Name: A, dtype: float64

You can use callable indexing in Series.

In [73]:
df13['A'].loc[lambda sx: sx > 0]

c    0.397711
f    1.922169
Name: A, dtype: float64

Using these methods / indexers, you can chain data selection operations without using a temporary variable.

In [74]:
bb13 = pd.read_csv('data/baseball.csv', index_col='id')
(bb13.groupby(['year', 'team']).sum().loc[lambda dfx: dfx['r'] > 100])

stint    g    ab    r    h  X2b  X3b  hr    rbi    sb   cs   bb  \
year team                                                                    
2007 CIN       6  379   745  101  203   35    2  36  125.0  10.0  1.0  105   
     DET       5  301  1062  162  283   54    4  37  144.0  24.0  7.0   97   
     HOU       4  311   926  109  218   47    6  14   77.0  10.0  4.0   60   
     LAN      11  413  1021  153  293   61    3  36  154.0   7.0  5.0  114   
     NYN      13  622  1854  240  509  101    3  61  243.0  22.0  4.0  174   
     SFN       5  482  1305  198  337   67    6  40  171.0  26.0  7.0  235   
     TEX       2  198   729  115  200   40    4  28  115.0  21.0  4.0   73   
     TOR       4  459  1408  187  378   96    2  58  223.0   4.0  2.0  190   

              so   ibb   hbp    sh    sf  gidp  
year team                                       
2007 CIN   127.0  14.0   1.0   1.0  15.0  18.0  
     DET   176.0   3.0  10.0   4.0   8.0  28.0  
     HOU   212.0   3.0   9.0  16.0   6.0  17.0  
     LAN   141.0   8.0   9.0   3.0   8.0  29.0  
     NYN   310.0  24.0  23.0  18.0  15.0  48.0  
     SFN   188.0  51.0   8.0  16.0   6.0  41.0  
     TEX   140.0   4.0   5.0   2.0   8.0  16.0  
     TOR   265.0  16.0  12.0   4.0  16.0  38.0

## Combining positional and label-based indexing
If you wish to get the 0th and the 2nd elements from the index in the ‘A’ column, you can do:

In [75]:
df14 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6]},
                   index=list('abc'))
df14

,A,B
a,1,4
b,2,5
c,3,6


In [76]:
df14.loc[df14.index[[0, 2]], 'A']

a    1
c    3
Name: A, dtype: int64

This can also be expressed using .iloc, by explicitly getting locations on the indexers, and using positional indexing to select things.

In [77]:
df14.iloc[[0, 2], df14.columns.get_loc('A')]

a    1
c    3
Name: A, dtype: int64

For getting multiple indexers, using .get_indexer:

In [78]:
df14.iloc[[0, 2], df14.columns.get_indexer(['A', 'B'])]

,A,B
a,1,4
c,3,6


## Indexing with list with missing labels is deprecated
>Warning
>
>Changed in version 1.0.0.
>
>Using .loc or [] with a list with one or more missing labels will no longer reindex, in favor of .reindex.

In prior versions, using .loc[list-of-labels] would work as long as at least 1 of the keys was found (otherwise it would raise a KeyError). This behavior was changed and will now raise a KeyError if at least one label is missing. The recommended alternative is to use .reindex().

For example.

In [79]:
s14 = pd.Series([1, 2, 3])
s14

0    1
1    2
2    3
dtype: int64

Selection with all keys found is unchanged.

In [80]:
s14.loc[[1, 2]]

1    2
2    3
dtype: int64

Previous behavior

In [81]:
try:
    s14.loc[[1, 2, 3]]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'Passing list-likes to .loc with any non-matching elements will raise \n'
    msg = msg + 'KeyError in the future, you can use .reindex() as an alternative. \n'
    msg = msg + 'FIX: use correct index range \n'
    print(msg)

sys.exc_info(): <class 'KeyError'> 
Passing list-likes to .loc with any non-matching elements will raise 
KeyError in the future, you can use .reindex() as an alternative. 
FIX: use correct index range 



See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike

### Reindexing
The idiomatic way to achieve selecting potentially not-found elements is via .reindex(). See also the section on reindexing.

In [82]:
s14.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

Alternatively, if you want to select only valid keys, the following is idiomatic and efficient; it is guaranteed to preserve the dtype of the selection.

In [83]:
labels = [1, 2, 3]
s14.loc[s14.index.intersection(labels)]

1    2
2    3
dtype: int64

Having a duplicated index will raise for a .reindex():

In [84]:
s14 = pd.Series(np.arange(4), index=['a', 'a', 'b', 'c'])
labels = ['c', 'd']

In [85]:
try:
    s14.reindex(labels)
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'ValueError: cannot reindex from a duplicate axis \n'
    msg = msg + 'FIX: only reindex primary index \n'
    print(msg)

sys.exc_info(): <class 'ValueError'> 
ValueError: cannot reindex from a duplicate axis 
FIX: only reindex primary index 



Generally, you can intersect the desired labels with the current axis, and then reindex.

In [86]:
s14.loc[s14.index.intersection(labels)].reindex(labels)

c    3.0
d    NaN
dtype: float64

However, this would still raise if your resulting index is duplicated.

In [87]:
try:
    s14.loc[s14.index.intersection(labels)].reindex(labels)
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'ValueError: cannot reindex from a duplicate axis \n'
    msg = msg + 'FIX: only reindex primary index \n'
    print(msg)

## Selecting random samples
A random selection of rows or columns from a Series or DataFrame with the sample() method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [88]:
s15 = pd.Series([0, 1, 2, 3, 4, 5])

# When no arguments are passed, returns 1 row.
s15.sample()

1    1
dtype: int64

In [89]:
# One may specify either a number of rows:
s15.sample(n=3)

1    1
5    5
2    2
dtype: int64

In [90]:
# Or a fraction of the rows:
s15.sample(frac=0.5)

2    2
0    0
4    4
dtype: int64

By default, sample will return each row at most once, but one can also sample with replacement using the replace option:

In [91]:
s16 = pd.Series([0, 1, 2, 3, 4, 5])
# Without replacement (default):
s16.sample(n=6, replace=False)

5    5
3    3
2    2
1    1
4    4
0    0
dtype: int64

In [92]:
# With replacement:
s16.sample(n=6, replace=True)

0    0
0    0
1    1
2    2
2    2
0    0
dtype: int64

By default, each row has an equal probability of being selected, but if you want rows to have different probabilities, you can pass the sample function sampling weights as weights. These weights can be a list, a NumPy array, or a Series, but they must be of the same length as the object you are sampling. Missing values will be treated as a weight of zero, and inf values are not allowed. If weights do not sum to 1, they will be re-normalized by dividing all weights by the sum of the weights. For example:

In [93]:
s17 = pd.Series([0, 1, 2, 3, 4, 5])
example_weights = [0, 0, 0.2, 0.2, 0.2, 0.4]
s17.sample(n=3, weights=example_weights)

5    5
3    3
2    2
dtype: int64

In [94]:
# Weights will be re-normalized automatically
example_weights2 = [0.5, 0, 0, 0, 0, 0]
s17.sample(n=1, weights=example_weights2)

0    0
dtype: int64

When applied to a DataFrame, you can use a column of the DataFrame as sampling weights (provided you are sampling rows and not columns) by simply passing the name of the column as a string.

In [95]:
df17 = pd.DataFrame({'col1': [9, 8, 7, 6],
                     'weight_column': [0.5, 0.4, 0.1, 0]})
df17.sample(n=3, weights='weight_column')

,col1,weight_column
0,9,0.5
1,8,0.4
2,7,0.1


sample also allows users to sample columns instead of rows using the axis argument.

In [96]:
df18 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})
df18.sample(n=1, axis=1)

,col2
0,2
1,3
2,4


Finally, one can also set a seed for sample’s random number generator using the random_state argument, which will accept either an integer (as a seed) or a NumPy RandomState object.

In [97]:
df19 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})

# With a given seed, the sample will always draw the same rows.
df19.sample(n=2, random_state=2)

,col1,col2
2,3,4
1,2,3


In [98]:
df19.sample(n=2, random_state=2)

,col1,col2
2,3,4
1,2,3


## Setting with enlargement
The .loc/[] operations can perform enlargement when setting a non-existent key for that axis.

In the Series case this is effectively an appending operation.

In [99]:
s20 = pd.Series([1, 2, 3])
s20

0    1
1    2
2    3
dtype: int64

In [100]:
s20[5] = 5.
s20

0    1.0
1    2.0
2    3.0
5    5.0
dtype: float64

A DataFrame can be enlarged on either axis via .loc.

In [101]:
df20 = pd.DataFrame(np.arange(6).reshape(3, 2),
                   columns=['A', 'B'])
df20

,A,B
0,0,1
1,2,3
2,4,5


In [102]:
df20.loc[:, 'C'] = df20.loc[:, 'A']
df20

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4


This is like an append operation on the DataFrame.

In [103]:
df20.loc[3] = 5
df20

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4
3,5,5,5


## Fast scalar value getting and setting
Since indexing with [] must handle a lot of cases (single-label access, slicing, boolean indexing, etc.), it has a bit of overhead in order to figure out what you’re asking for. If you only want to access a scalar value, the fastest way is to use the at and iat methods, which are implemented on all of the data structures.

Similarly to loc, at provides label based scalar lookups, while, iat provides integer based lookups analogously to iloc

In [104]:
s17.iat[5]

5

In [105]:
df21 = df1.copy()
df21.at[dates[5], 'A']

0.17362502462808632

In [106]:
df21.iat[3, 0]

0.2558142075735626

You can also set using these same indexers.

In [107]:
df21.at[dates[5], 'E'] = 7
df21.iat[3, 0] = 7

at may enlarge the object in-place as above if the indexer is missing.

In [108]:
df21.at[dates[-1] + pd.Timedelta('1 day'), 0] = 7
df21

,A,B,C,D,E,0
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438,NaN,NaN
2000-01-02,2.021892,0.070927,0.800103,0.118657,NaN,NaN
2000-01-03,-2.040983,0.098607,-1.503086,0.420989,NaN,NaN
2000-01-04,7.000000,-1.303977,0.954433,-1.137952,NaN,NaN
2000-01-05,0.020985,0.476644,0.850724,-0.722163,NaN,NaN
2000-01-06,0.173625,-0.699478,0.355988,1.610516,7.0,NaN
2000-01-07,0.880987,-1.717641,0.809490,1.449279,NaN,NaN
2000-01-08,-1.805341,0.762664,-0.205993,-1.595724,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,7.0


## Boolean indexing
Another common operation is the use of boolean vectors to filter the data. The operators are: | for or, & for and, and ~ for not. These must be grouped by using parentheses, since by default Python will evaluate an expression such as df['A'] > 2 & df['B'] < 3 as df['A'] > (2 & df['B']) < 3, while the desired evaluation order is (df['A'] > 2) & (df['B'] < 3).

Using a boolean vector to index a Series works exactly as in a NumPy ndarray:

In [109]:
s22 = pd.Series(range(-3, 4))
s22

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [110]:
s22[s22 > 0]

4    1
5    2
6    3
dtype: int64

In [111]:
s22[(s22 < -1) | (s22 > 0.5)]

0   -3
1   -2
4    1
5    2
6    3
dtype: int64

In [112]:
s22[~(s22 < 0)]

3    0
4    1
5    2
6    3
dtype: int64

You may select rows from a DataFrame using a boolean vector the same length as the DataFrame’s index (for example, something derived from one of the columns of the DataFrame):

In [113]:
df22 = df21.copy()
df22[df22['A'] > 0]

,A,B,C,D,E,0
2000-01-02,2.021892,0.070927,0.800103,0.118657,NaN,NaN
2000-01-04,7.000000,-1.303977,0.954433,-1.137952,NaN,NaN
2000-01-05,0.020985,0.476644,0.850724,-0.722163,NaN,NaN
2000-01-06,0.173625,-0.699478,0.355988,1.610516,7.0,NaN
2000-01-07,0.880987,-1.717641,0.809490,1.449279,NaN,NaN


List comprehensions and the map method of Series can also be used to produce more complex criteria:

In [114]:
df23 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                     'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                     'c': np.random.randn(7)})

# only want 'two' or 'three'
criterion = df23['a'].map(lambda x: x.startswith('t'))
df23[criterion]

,a,b,c
2,two,y,-0.678364
3,three,x,-0.827769
4,two,y,-0.366825


In [115]:
# equivalent but slower
df23[[x.startswith('t') for x in df23['a']]]

,a,b,c
2,two,y,-0.678364
3,three,x,-0.827769
4,two,y,-0.366825


In [116]:
# Multiple criteria
df23[criterion & (df23['b'] == 'x')]

,a,b,c
3,three,x,-0.827769


With the choice methods Selection by Label, Selection by Position, and Advanced Indexing you may select along more than one axis using boolean vectors combined with other indexing expressions.

In [117]:
df23.loc[criterion & (df23['b'] == 'x'), 'b':'c']

,b,c
3,x,-0.827769


>Warning
>
>iloc supports two kinds of boolean indexing. If the indexer is a boolean Series, an error will be raised. For instance, in the following example, df.iloc[s.values, 1] is ok. The boolean indexer is an array. But df.iloc[s, 1] would raise ValueError.

In [118]:
df24 = pd.DataFrame([[1, 2], [3, 4], [5, 6]],
                    index=list('abc'),
                    columns=['A', 'B'])
s24 = (df24['A'] > 2)
s24

a    False
b     True
c     True
Name: A, dtype: bool

In [119]:
df24.loc[s24, 'B']

b    4
c    6
Name: B, dtype: int64

In [120]:
df24.iloc[s24.values, 1]

b    4
c    6
Name: B, dtype: int64

## Indexing with isin
Consider the isin() method of Series, which returns a boolean vector that is true wherever the Series elements exist in the passed list. This allows you to select rows where one or more columns have values you want:

In [121]:
s25 = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s25

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [122]:
s25.isin([2, 4, 6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [123]:
s25[s25.isin([2, 4, 6])]

2    2
0    4
dtype: int64

The same method is available for Index objects and is useful for the cases when you don’t know which of the sought labels are in fact present:

In [124]:
s25[s25.index.isin([2, 4, 6])]

4    0
2    2
dtype: int64

In [125]:
# compare it to the following
s25.reindex([2, 4, 6])

2    2.0
4    0.0
6    NaN
dtype: float64

In addition to that, MultiIndex allows selecting a separate level to use in the membership check:

In [126]:
s26 = pd.Series(np.arange(6),
                 index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s26

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int64

In [127]:
s26.iloc[s26.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]

0  c    2
1  a    3
dtype: int64

In [128]:
s26.iloc[s26.index.isin(['a', 'c', 'e'], level=1)]

0  a    0
   c    2
1  a    3
   c    5
dtype: int64

DataFrame also has an isin() method. When calling isin, pass a set of values as either an array or dict. If values is an array, isin returns a DataFrame of booleans that is the same shape as the original DataFrame, with True wherever the element is in the sequence of values.

In [129]:
df27 = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                     'ids2': ['a', 'n', 'c', 'n']})
values = ['a', 'b', 1, 3]
df27.isin(values)

,vals,ids,ids2
0,True,True,True
1,False,True,False
2,True,False,False
3,False,False,False


Oftentimes you’ll want to match certain values with certain columns. Just make values a dict where the key is the column, and the value is a list of items you want to check for.

In [130]:
values = {'ids': ['a', 'b'], 'vals': [1, 3]}
df27.isin(values)

,vals,ids,ids2
0,True,True,False
1,False,True,False
2,True,False,False
3,False,False,False


Combine DataFrame’s isin with the any() and all() methods to quickly select subsets of your data that meet a given criteria. To select a row where each column meets its own criterion:

In [131]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}
row_mask = df27.isin(values).all(1)
df27[row_mask]

,vals,ids,ids2
0,1,a,a


## The where() Method and Masking
Selecting values from a Series with a boolean vector generally returns a subset of the data. To guarantee that selection output has the same shape as the original data, you can use the where method in Series and DataFrame.

To return only the selected rows:

In [132]:
s25[s25 > 0]

3    1
2    2
1    3
0    4
dtype: int64

To return a Series of the same shape as the original:

In [133]:
s25.where(s25 > 0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

Selecting values from a DataFrame with a boolean criterion now also preserves input data shape. where is used under the hood as the implementation. The code below is equivalent to df.where(df < 0).

In [134]:
df1[df1 < 0]

,A,B,C,D
2000-01-01,-0.119524,-1.139756,NaN,-0.285438
2000-01-02,NaN,NaN,NaN,NaN
2000-01-03,-2.040983,NaN,-1.503086,NaN
2000-01-04,NaN,-1.303977,NaN,-1.137952
2000-01-05,NaN,NaN,NaN,-0.722163
2000-01-06,NaN,-0.699478,NaN,NaN
2000-01-07,NaN,-1.717641,NaN,NaN
2000-01-08,-1.805341,NaN,-0.205993,-1.595724


In addition, where takes an optional other argument for replacement of values where the condition is False, in the returned copy.

In [135]:
df1.where(df1 < 0, -df1)

,A,B,C,D
2000-01-01,-0.119524,-1.139756,-0.517904,-0.285438
2000-01-02,-2.021892,-0.070927,-0.800103,-0.118657
2000-01-03,-2.040983,-0.098607,-1.503086,-0.420989
2000-01-04,-0.255814,-1.303977,-0.954433,-1.137952
2000-01-05,-0.020985,-0.476644,-0.850724,-0.722163
2000-01-06,-0.173625,-0.699478,-0.355988,-1.610516
2000-01-07,-0.880987,-1.717641,-0.809490,-1.449279
2000-01-08,-1.805341,-0.762664,-0.205993,-1.595724


You may wish to set values based on some boolean criteria. This can be done intuitively like so:

In [136]:
s26 = s25.copy()
s26[s26 < 0] = 0
s26

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [137]:
df26 = df1.copy()
df26[df26 < 0] = 0
df26

,A,B,C,D
2000-01-01,0.000000,0.000000,0.517904,0.000000
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,0.000000,0.098607,0.000000,0.420989
2000-01-04,0.255814,0.000000,0.954433,0.000000
2000-01-05,0.020985,0.476644,0.850724,0.000000
2000-01-06,0.173625,0.000000,0.355988,1.610516
2000-01-07,0.880987,0.000000,0.809490,1.449279
2000-01-08,0.000000,0.762664,0.000000,0.000000


By default, where returns a modified copy of the data. There is an optional parameter inplace so that the original data can be modified without creating a copy:

In [138]:
df27 = df1.copy()
df27.where(df1 > 0, -df1, inplace=True)
df27

,A,B,C,D
2000-01-01,0.119524,1.139756,0.517904,0.285438
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,2.040983,0.098607,1.503086,0.420989
2000-01-04,0.255814,1.303977,0.954433,1.137952
2000-01-05,0.020985,0.476644,0.850724,0.722163
2000-01-06,0.173625,0.699478,0.355988,1.610516
2000-01-07,0.880987,1.717641,0.809490,1.449279
2000-01-08,1.805341,0.762664,0.205993,1.595724


>Note
>
>The signature for DataFrame.where() differs from numpy.where(). Roughly df1.where(m, df2) is equivalent to np.where(m, df1, df2).

In [139]:
df1.where(df1 < 0, -df1) == np.where(df1 < 0, df1, -df1)

,A,B,C,D
2000-01-01,True,True,True,True
2000-01-02,True,True,True,True
2000-01-03,True,True,True,True
2000-01-04,True,True,True,True
2000-01-05,True,True,True,True
2000-01-06,True,True,True,True
2000-01-07,True,True,True,True
2000-01-08,True,True,True,True


#### Alignment

Furthermore, where aligns the input boolean condition (ndarray or DataFrame), such that partial selection with setting is possible. This is analogous to partial setting via .loc (but on the contents rather than the axis labels).

In [140]:
df28 = df1.copy()
df28[df28[1:4] > 0] = 3
df28

,A,B,C,D
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438
2000-01-02,3.000000,3.000000,3.000000,3.000000
2000-01-03,-2.040983,3.000000,-1.503086,3.000000
2000-01-04,3.000000,-1.303977,3.000000,-1.137952
2000-01-05,0.020985,0.476644,0.850724,-0.722163
2000-01-06,0.173625,-0.699478,0.355988,1.610516
2000-01-07,0.880987,-1.717641,0.809490,1.449279
2000-01-08,-1.805341,0.762664,-0.205993,-1.595724


Where can also accept axis and level parameters to align the input when performing the where.

In [141]:
df28 = df1.copy()
df29 = df1.copy()
df28

,A,B,C,D
2000-01-01,-0.119524,-1.139756,0.517904,-0.285438
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,-2.040983,0.098607,-1.503086,0.420989
2000-01-04,0.255814,-1.303977,0.954433,-1.137952
2000-01-05,0.020985,0.476644,0.850724,-0.722163
2000-01-06,0.173625,-0.699478,0.355988,1.610516
2000-01-07,0.880987,-1.717641,0.809490,1.449279
2000-01-08,-1.805341,0.762664,-0.205993,-1.595724


In [142]:
df28.where(df28 > 0, df28['A'], axis='index')

,A,B,C,D
2000-01-01,-0.119524,-0.119524,0.517904,-0.119524
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,-2.040983,0.098607,-2.040983,0.420989
2000-01-04,0.255814,0.255814,0.954433,0.255814
2000-01-05,0.020985,0.476644,0.850724,0.020985
2000-01-06,0.173625,0.173625,0.355988,1.610516
2000-01-07,0.880987,0.880987,0.809490,1.449279
2000-01-08,-1.805341,0.762664,-1.805341,-1.805341


This is equivalent to (but faster than) the following.

In [143]:
df29.apply(lambda x, y: x.where(x > 0, y), y=df29['A'])

,A,B,C,D
2000-01-01,-0.119524,-0.119524,0.517904,-0.119524
2000-01-02,2.021892,0.070927,0.800103,0.118657
2000-01-03,-2.040983,0.098607,-2.040983,0.420989
2000-01-04,0.255814,0.255814,0.954433,0.255814
2000-01-05,0.020985,0.476644,0.850724,0.020985
2000-01-06,0.173625,0.173625,0.355988,1.610516
2000-01-07,0.880987,0.880987,0.809490,1.449279
2000-01-08,-1.805341,0.762664,-1.805341,-1.805341


where can accept a callable as condition and other arguments. The function must be with one argument (the calling Series or DataFrame) and that returns valid output as condition and other argument.

In [144]:
df30 = pd.DataFrame({'A': [1, 2, 3],
                     'B': [4, 5, 6],
                     'C': [7, 8, 9]})
df30.where(lambda x: x > 4, lambda x: x + 10)

,A,B,C
0,11,14,7
1,12,5,8
2,13,6,9


### Mask
mask() is the inverse boolean operation of where.

In [145]:
s25.mask(s25 >= 0)

4   NaN
3   NaN
2   NaN
1   NaN
0   NaN
dtype: float64

In [146]:
df1.mask(df1 >= 0)

,A,B,C,D
2000-01-01,-0.119524,-1.139756,NaN,-0.285438
2000-01-02,NaN,NaN,NaN,NaN
2000-01-03,-2.040983,NaN,-1.503086,NaN
2000-01-04,NaN,-1.303977,NaN,-1.137952
2000-01-05,NaN,NaN,NaN,-0.722163
2000-01-06,NaN,-0.699478,NaN,NaN
2000-01-07,NaN,-1.717641,NaN,NaN
2000-01-08,-1.805341,NaN,-0.205993,-1.595724


## Setting with enlargement conditionally using numpy()
An alternative to where() is to use numpy.where(). Combined with setting a new column, you can use it to enlarge a dataframe where the values are determined conditionally.

Consider you have two choices to choose from in the following dataframe. And you want to set a new column color to ‘green’ when the second column has ‘Z’. You can do the following:

In [147]:
df32 = pd.DataFrame({'col1': list('ABBC'), 'col2': list('ZZXY')})
df32['color'] = np.where(df32['col2'] == 'Z', 'green', 'red')
df32

,col1,col2,color
0,A,Z,green
1,B,Z,green
2,B,X,red
3,C,Y,red


If you have multiple conditions, you can use numpy.select() to achieve that. Say corresponding to three conditions there are three choice of colors, with a fourth color as a fallback, you can do the following.

In [148]:
conditions = [
    (df32['col2'] == 'Z') & (df32['col1'] == 'A'),
    (df32['col2'] == 'Z') & (df32['col1'] == 'B'),
    (df32['col1'] == 'B')
]

choices = ['yellow', 'blue', 'purple']
df32['color'] = np.select(conditions, choices, default='black')
df32

,col1,col2,color
0,A,Z,yellow
1,B,Z,blue
2,B,X,purple
3,C,Y,black


## The query() Method
DataFrame objects have a query() method that allows selection using an expression.

You can get the value of the frame where column b has values between the values of columns a and c. For example:

In [149]:
n = 10
df33 = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df33

,a,b,c
0,0.665938,0.489127,0.430012
1,0.141150,0.808962,0.450953
2,0.124912,0.357459,0.586972
3,0.514371,0.989672,0.437663
4,0.714716,0.219849,0.259279
5,0.262067,0.147357,0.956111
6,0.443938,0.930496,0.124194
7,0.606365,0.377903,0.941993
8,0.589767,0.387538,0.194538
9,0.981236,0.393476,0.229179


In [150]:
# pure python
df33[(df33['a'] < df33['b']) & (df33['b'] < df33['c'])]

,a,b,c
2,0.124912,0.357459,0.586972


In [151]:
# query
df33.query('(a < b) & (b < c)')

,a,b,c
2,0.124912,0.357459,0.586972


Do the same thing but fall back on a named index if there is no column with the name a.

In [152]:
df34 = pd.DataFrame(np.random.randint(n / 2, size=(n, 2)), columns=list('bc'))
df34.index.name = 'a'
df34

,b,c
a,,
0,0,1
1,2,1
2,4,2
3,4,2
4,2,1
5,4,3
6,0,4
7,2,1
8,2,0


In [153]:
df34.query('a < b and b < c')

,b,c
a,,


If instead you don’t want to or cannot name your index, you can use the name index in your query expression:

In [154]:
df35 = pd.DataFrame(np.random.randint(n, size=(n, 2)), columns=list('bc'))
df35

,b,c
0,3,0
1,6,5
2,0,8
3,0,3
4,8,2
5,5,4
6,2,4
7,0,8
8,0,7
9,7,3


In [155]:
df35.query('index < b < c')

,b,c


>Note
>
>If the name of your index overlaps with a column name, the column name is given precedence. For example,

In [156]:
df36 = pd.DataFrame({'a': np.random.randint(5, size=5)})
df36.index.name = 'a'
df36.query('a > 2')  # uses the column 'a', not the index

,a
a,
0,4
1,3
2,3
3,3


>You can still use the index in a query expression by using the special identifier ‘index’:

In [157]:
df36.query('index > 2')

,a
a,
3,3
4,1


>If for some reason you have a column named index, then you can refer to the index as ilevel_0 as well, but at this point you should consider renaming your columns to something less ambiguous.

## MultiIndex query() Syntax
You can also use the levels of a DataFrame with a MultiIndex as if they were columns in the frame:

In [158]:
n = 10
colors = np.random.choice(['red', 'green'], size=n)
foods = np.random.choice(['eggs', 'ham'], size=n)
colors

array(['red', 'green', 'green', 'red', 'red', 'green', 'red', 'red',
       'green', 'red'], dtype='<U5')



In [231]: 
array(['red', 'red', 'red', 'green', 'green', 'green', 'green', 'green',
       'green', 'green'], dtype='<U5')

In [235]: foods
Out[235]: 
array(['ham', 'ham', 'eggs', 'eggs', 'eggs', 'ham', 'ham', 'eggs', 'eggs',
       'eggs'], dtype='<U4')

In [236]: index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])

In [237]: df = pd.DataFrame(np.random.randn(n, 2), index=index)

In [238]: df
Out[238]: 
                   0         1
color food                    
red   ham   0.194889 -0.381994
      ham   0.318587  2.089075
      eggs -0.728293 -0.090255
green eggs -0.748199  1.318931
      eggs -2.029766  0.792652
      ham   0.461007 -0.542749
      ham  -0.305384 -0.479195
      eggs  0.095031 -0.270099
      eggs -0.707140 -0.773882
      eggs  0.229453  0.304418

In [239]: df.query('color == "red"')
Out[239]: 
                   0         1
color food                    
red   ham   0.194889 -0.381994
      ham   0.318587  2.089075
      eggs -0.728293 -0.090255
If the levels of the MultiIndex are unnamed, you can refer to them using special names:

In [240]: df.index.names = [None, None]

In [241]: df
Out[241]: 
                   0         1
red   ham   0.194889 -0.381994
      ham   0.318587  2.089075
      eggs -0.728293 -0.090255
green eggs -0.748199  1.318931
      eggs -2.029766  0.792652
      ham   0.461007 -0.542749
      ham  -0.305384 -0.479195
      eggs  0.095031 -0.270099
      eggs -0.707140 -0.773882
      eggs  0.229453  0.304418

In [242]: df.query('ilevel_0 == "red"')
Out[242]: 
                 0         1
red ham   0.194889 -0.381994
    ham   0.318587  2.089075
    eggs -0.728293 -0.090255
The convention is ilevel_0, which means “index level 0” for the 0th level of the index.

query() Use Cases
A use case for query() is when you have a collection of DataFrame objects that have a subset of column names (or index levels/names) in common. You can pass the same query to both frames without having to specify which frame you’re interested in querying

In [243]: df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))

In [244]: df
Out[244]: 
          a         b         c
0  0.224283  0.736107  0.139168
1  0.302827  0.657803  0.713897
2  0.611185  0.136624  0.984960
3  0.195246  0.123436  0.627712
4  0.618673  0.371660  0.047902
5  0.480088  0.062993  0.185760
6  0.568018  0.483467  0.445289
7  0.309040  0.274580  0.587101
8  0.258993  0.477769  0.370255
9  0.550459  0.840870  0.304611

In [245]: df2 = pd.DataFrame(np.random.rand(n + 2, 3), columns=df.columns)

In [246]: df2
Out[246]: 
           a         b         c
0   0.357579  0.229800  0.596001
1   0.309059  0.957923  0.965663
2   0.123102  0.336914  0.318616
3   0.526506  0.323321  0.860813
4   0.518736  0.486514  0.384724
5   0.190804  0.505723  0.614533
6   0.891939  0.623977  0.676639
7   0.480559  0.378528  0.460858
8   0.420223  0.136404  0.141295
9   0.732206  0.419540  0.604675
10  0.604466  0.848974  0.896165
11  0.589168  0.920046  0.732716

In [247]: expr = '0.0 <= a <= c <= 0.5'

In [248]: map(lambda frame: frame.query(expr), [df, df2])
Out[248]: <map at 0x7fc1d96ba4c0>
query() Python versus pandas Syntax Comparison
Full numpy-like syntax:

In [249]: df = pd.DataFrame(np.random.randint(n, size=(n, 3)), columns=list('abc'))

In [250]: df
Out[250]: 
   a  b  c
0  7  8  9
1  1  0  7
2  2  7  2
3  6  2  2
4  2  6  3
5  3  8  2
6  1  7  2
7  5  1  5
8  9  8  0
9  1  5  0

In [251]: df.query('(a < b) & (b < c)')
Out[251]: 
   a  b  c
0  7  8  9

In [252]: df[(df['a'] < df['b']) & (df['b'] < df['c'])]
Out[252]: 
   a  b  c
0  7  8  9
Slightly nicer by removing the parentheses (by binding making comparison operators bind tighter than & and |).

In [253]: df.query('a < b & b < c')
Out[253]: 
   a  b  c
0  7  8  9
Use English instead of symbols:

In [254]: df.query('a < b and b < c')
Out[254]: 
   a  b  c
0  7  8  9
Pretty close to how you might write it on paper:

In [255]: df.query('a < b < c')
Out[255]: 
   a  b  c
0  7  8  9
The in and not in operators
query() also supports special use of Python’s in and not in comparison operators, providing a succinct syntax for calling the isin method of a Series or DataFrame.

# get all rows where columns "a" and "b" have overlapping values
In [256]: df = pd.DataFrame({'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
   .....:                    'c': np.random.randint(5, size=12),
   .....:                    'd': np.random.randint(9, size=12)})
   .....: 

In [257]: df
Out[257]: 
    a  b  c  d
0   a  a  2  6
1   a  a  4  7
2   b  a  1  6
3   b  a  2  1
4   c  b  3  6
5   c  b  0  2
6   d  b  3  3
7   d  b  2  1
8   e  c  4  3
9   e  c  2  0
10  f  c  0  6
11  f  c  1  2

In [258]: df.query('a in b')
Out[258]: 
   a  b  c  d
0  a  a  2  6
1  a  a  4  7
2  b  a  1  6
3  b  a  2  1
4  c  b  3  6
5  c  b  0  2

# How you'd do it in pure Python
In [259]: df[df['a'].isin(df['b'])]
Out[259]: 
   a  b  c  d
0  a  a  2  6
1  a  a  4  7
2  b  a  1  6
3  b  a  2  1
4  c  b  3  6
5  c  b  0  2

In [260]: df.query('a not in b')
Out[260]: 
    a  b  c  d
6   d  b  3  3
7   d  b  2  1
8   e  c  4  3
9   e  c  2  0
10  f  c  0  6
11  f  c  1  2

# pure Python
In [261]: df[~df['a'].isin(df['b'])]
Out[261]: 
    a  b  c  d
6   d  b  3  3
7   d  b  2  1
8   e  c  4  3
9   e  c  2  0
10  f  c  0  6
11  f  c  1  2
You can combine this with other expressions for very succinct queries:

# rows where cols a and b have overlapping values
# and col c's values are less than col d's
In [262]: df.query('a in b and c < d')
Out[262]: 
   a  b  c  d
0  a  a  2  6
1  a  a  4  7
2  b  a  1  6
4  c  b  3  6
5  c  b  0  2

# pure Python
In [263]: df[df['b'].isin(df['a']) & (df['c'] < df['d'])]
Out[263]: 
    a  b  c  d
0   a  a  2  6
1   a  a  4  7
2   b  a  1  6
4   c  b  3  6
5   c  b  0  2
10  f  c  0  6
11  f  c  1  2
Note

Note that in and not in are evaluated in Python, since numexpr has no equivalent of this operation. However, only the in/not in expression itself is evaluated in vanilla Python. For example, in the expression

df.query('a in b + c + d')
(b + c + d) is evaluated by numexpr and then the in operation is evaluated in plain Python. In general, any operations that can be evaluated using numexpr will be.

Special use of the == operator with list objects
Comparing a list of values to a column using ==/!= works similarly to in/not in.

In [264]: df.query('b == ["a", "b", "c"]')
Out[264]: 
    a  b  c  d
0   a  a  2  6
1   a  a  4  7
2   b  a  1  6
3   b  a  2  1
4   c  b  3  6
5   c  b  0  2
6   d  b  3  3
7   d  b  2  1
8   e  c  4  3
9   e  c  2  0
10  f  c  0  6
11  f  c  1  2

# pure Python
In [265]: df[df['b'].isin(["a", "b", "c"])]
Out[265]: 
    a  b  c  d
0   a  a  2  6
1   a  a  4  7
2   b  a  1  6
3   b  a  2  1
4   c  b  3  6
5   c  b  0  2
6   d  b  3  3
7   d  b  2  1
8   e  c  4  3
9   e  c  2  0
10  f  c  0  6
11  f  c  1  2

In [266]: df.query('c == [1, 2]')
Out[266]: 
    a  b  c  d
0   a  a  2  6
2   b  a  1  6
3   b  a  2  1
7   d  b  2  1
9   e  c  2  0
11  f  c  1  2

In [267]: df.query('c != [1, 2]')
Out[267]: 
    a  b  c  d
1   a  a  4  7
4   c  b  3  6
5   c  b  0  2
6   d  b  3  3
8   e  c  4  3
10  f  c  0  6

# using in/not in
In [268]: df.query('[1, 2] in c')
Out[268]: 
    a  b  c  d
0   a  a  2  6
2   b  a  1  6
3   b  a  2  1
7   d  b  2  1
9   e  c  2  0
11  f  c  1  2

In [269]: df.query('[1, 2] not in c')
Out[269]: 
    a  b  c  d
1   a  a  4  7
4   c  b  3  6
5   c  b  0  2
6   d  b  3  3
8   e  c  4  3
10  f  c  0  6

# pure Python
In [270]: df[df['c'].isin([1, 2])]
Out[270]: 
    a  b  c  d
0   a  a  2  6
2   b  a  1  6
3   b  a  2  1
7   d  b  2  1
9   e  c  2  0
11  f  c  1  2
Boolean operators
You can negate boolean expressions with the word not or the ~ operator.

In [271]: df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))

In [272]: df['bools'] = np.random.rand(len(df)) > 0.5

In [273]: df.query('~bools')
Out[273]: 
          a         b         c  bools
2  0.697753  0.212799  0.329209  False
7  0.275396  0.691034  0.826619  False
8  0.190649  0.558748  0.262467  False

In [274]: df.query('not bools')
Out[274]: 
          a         b         c  bools
2  0.697753  0.212799  0.329209  False
7  0.275396  0.691034  0.826619  False
8  0.190649  0.558748  0.262467  False

In [275]: df.query('not bools') == df[~df['bools']]
Out[275]: 
      a     b     c  bools
2  True  True  True   True
7  True  True  True   True
8  True  True  True   True
Of course, expressions can be arbitrarily complex too:

# short query syntax
In [276]: shorter = df.query('a < b < c and (not bools) or bools > 2')

# equivalent in pure Python
In [277]: longer = df[(df['a'] < df['b'])
   .....:             & (df['b'] < df['c'])
   .....:             & (~df['bools'])
   .....:             | (df['bools'] > 2)]
   .....: 

In [278]: shorter
Out[278]: 
          a         b         c  bools
7  0.275396  0.691034  0.826619  False

In [279]: longer
Out[279]: 
          a         b         c  bools
7  0.275396  0.691034  0.826619  False

In [280]: shorter == longer
Out[280]: 
      a     b     c  bools
7  True  True  True   True
Performance of query()
DataFrame.query() using numexpr is slightly faster than Python for large frames.

../_images/query-perf.png
Note

You will only see the performance benefits of using the numexpr engine with DataFrame.query() if your frame has more than approximately 200,000 rows.

../_images/query-perf-small.png
This plot was created using a DataFrame with 3 columns each containing floating point values generated using numpy.random.randn().

Duplicate data
If you want to identify and remove duplicate rows in a DataFrame, there are two methods that will help: duplicated and drop_duplicates. Each takes as an argument the columns to use to identify duplicated rows.

duplicated returns a boolean vector whose length is the number of rows, and which indicates whether a row is duplicated.

drop_duplicates removes duplicate rows.

By default, the first observed row of a duplicate set is considered unique, but each method has a keep parameter to specify targets to be kept.

keep='first' (default): mark / drop duplicates except for the first occurrence.

keep='last': mark / drop duplicates except for the last occurrence.

keep=False: mark / drop all duplicates.

In [281]: df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
   .....:                     'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'],
   .....:                     'c': np.random.randn(7)})
   .....: 

In [282]: df2
Out[282]: 
       a  b         c
0    one  x -1.067137
1    one  y  0.309500
2    two  x -0.211056
3    two  y -1.842023
4    two  x -0.390820
5  three  x -1.964475
6   four  x  1.298329

In [283]: df2.duplicated('a')
Out[283]: 
0    False
1     True
2    False
3     True
4     True
5    False
6    False
dtype: bool

In [284]: df2.duplicated('a', keep='last')
Out[284]: 
0     True
1    False
2     True
3     True
4    False
5    False
6    False
dtype: bool

In [285]: df2.duplicated('a', keep=False)
Out[285]: 
0     True
1     True
2     True
3     True
4     True
5    False
6    False
dtype: bool

In [286]: df2.drop_duplicates('a')
Out[286]: 
       a  b         c
0    one  x -1.067137
2    two  x -0.211056
5  three  x -1.964475
6   four  x  1.298329

In [287]: df2.drop_duplicates('a', keep='last')
Out[287]: 
       a  b         c
1    one  y  0.309500
4    two  x -0.390820
5  three  x -1.964475
6   four  x  1.298329

In [288]: df2.drop_duplicates('a', keep=False)
Out[288]: 
       a  b         c
5  three  x -1.964475
6   four  x  1.298329
Also, you can pass a list of columns to identify duplications.

In [289]: df2.duplicated(['a', 'b'])
Out[289]: 
0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [290]: df2.drop_duplicates(['a', 'b'])
Out[290]: 
       a  b         c
0    one  x -1.067137
1    one  y  0.309500
2    two  x -0.211056
3    two  y -1.842023
5  three  x -1.964475
6   four  x  1.298329
To drop duplicates by index value, use Index.duplicated then perform slicing. The same set of options are available for the keep parameter.

In [291]: df3 = pd.DataFrame({'a': np.arange(6),
   .....:                     'b': np.random.randn(6)},
   .....:                    index=['a', 'a', 'b', 'c', 'b', 'a'])
   .....: 

In [292]: df3
Out[292]: 
   a         b
a  0  1.440455
a  1  2.456086
b  2  1.038402
c  3 -0.894409
b  4  0.683536
a  5  3.082764

In [293]: df3.index.duplicated()
Out[293]: array([False,  True, False, False,  True,  True])

In [294]: df3[~df3.index.duplicated()]
Out[294]: 
   a         b
a  0  1.440455
b  2  1.038402
c  3 -0.894409

In [295]: df3[~df3.index.duplicated(keep='last')]
Out[295]: 
   a         b
c  3 -0.894409
b  4  0.683536
a  5  3.082764

In [296]: df3[~df3.index.duplicated(keep=False)]
Out[296]: 
   a         b
c  3 -0.894409
Dictionary-like get() method
Each of Series or DataFrame have a get method which can return a default value.

In [297]: s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])

In [298]: s.get('a')  # equivalent to s['a']
Out[298]: 1

In [299]: s.get('x', default=-1)
Out[299]: -1
Looking up values by index/column labels
Sometimes you want to extract a set of values given a sequence of row labels and column labels, this can be achieved by DataFrame.melt combined by filtering the corresponding rows with DataFrame.loc. For instance:

In [300]: df = pd.DataFrame({'col': ["A", "A", "B", "B"],
   .....:                    'A': [80, 23, np.nan, 22],
   .....:                    'B': [80, 55, 76, 67]})
   .....: 

In [301]: df
Out[301]: 
  col     A   B
0   A  80.0  80
1   A  23.0  55
2   B   NaN  76
3   B  22.0  67

In [302]: melt = df.melt('col')

In [303]: melt = melt.loc[melt['col'] == melt['variable'], 'value']

In [304]: melt.reset_index(drop=True)
Out[304]: 
0    80.0
1    23.0
2    76.0
3    67.0
Name: value, dtype: float64
Formerly this could be achieved with the dedicated DataFrame.lookup method which was deprecated in version 1.2.0.

Index objects
The pandas Index class and its subclasses can be viewed as implementing an ordered multiset. Duplicates are allowed. However, if you try to convert an Index object with duplicate entries into a set, an exception will be raised.

Index also provides the infrastructure necessary for lookups, data alignment, and reindexing. The easiest way to create an Index directly is to pass a list or other sequence to Index:

In [305]: index = pd.Index(['e', 'd', 'a', 'b'])

In [306]: index
Out[306]: Index(['e', 'd', 'a', 'b'], dtype='object')

In [307]: 'd' in index
Out[307]: True
You can also pass a name to be stored in the index:

In [308]: index = pd.Index(['e', 'd', 'a', 'b'], name='something')

In [309]: index.name
Out[309]: 'something'
The name, if set, will be shown in the console display:

In [310]: index = pd.Index(list(range(5)), name='rows')

In [311]: columns = pd.Index(['A', 'B', 'C'], name='cols')

In [312]: df = pd.DataFrame(np.random.randn(5, 3), index=index, columns=columns)

In [313]: df
Out[313]: 
cols         A         B         C
rows                              
0     1.295989 -1.051694  1.340429
1    -2.366110  0.428241  0.387275
2     0.433306  0.929548  0.278094
3     2.154730 -0.315628  0.264223
4     1.126818  1.132290 -0.353310

In [314]: df['A']
Out[314]: 
rows
0    1.295989
1   -2.366110
2    0.433306
3    2.154730
4    1.126818
Name: A, dtype: float64
Setting metadata
Indexes are “mostly immutable”, but it is possible to set and change their name attribute. You can use the rename, set_names to set these attributes directly, and they default to returning a copy.

See Advanced Indexing for usage of MultiIndexes.

In [315]: ind = pd.Index([1, 2, 3])

In [316]: ind.rename("apple")
Out[316]: Int64Index([1, 2, 3], dtype='int64', name='apple')

In [317]: ind
Out[317]: Int64Index([1, 2, 3], dtype='int64')

In [318]: ind.set_names(["apple"], inplace=True)

In [319]: ind.name = "bob"

In [320]: ind
Out[320]: Int64Index([1, 2, 3], dtype='int64', name='bob')
set_names, set_levels, and set_codes also take an optional level argument

In [321]: index = pd.MultiIndex.from_product([range(3), ['one', 'two']], names=['first', 'second'])

In [322]: index
Out[322]: 
MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two'),
            (2, 'one'),
            (2, 'two')],
           names=['first', 'second'])

In [323]: index.levels[1]
Out[323]: Index(['one', 'two'], dtype='object', name='second')

In [324]: index.set_levels(["a", "b"], level=1)
Out[324]: 
MultiIndex([(0, 'a'),
            (0, 'b'),
            (1, 'a'),
            (1, 'b'),
            (2, 'a'),
            (2, 'b')],
           names=['first', 'second'])
Set operations on Index objects
The two main operations are union and intersection. Difference is provided via the .difference() method.

In [325]: a = pd.Index(['c', 'b', 'a'])

In [326]: b = pd.Index(['c', 'e', 'd'])

In [327]: a.difference(b)
Out[327]: Index(['a', 'b'], dtype='object')
Also available is the symmetric_difference operation, which returns elements that appear in either idx1 or idx2, but not in both. This is equivalent to the Index created by idx1.difference(idx2).union(idx2.difference(idx1)), with duplicates dropped.

In [328]: idx1 = pd.Index([1, 2, 3, 4])

In [329]: idx2 = pd.Index([2, 3, 4, 5])

In [330]: idx1.symmetric_difference(idx2)
Out[330]: Int64Index([1, 5], dtype='int64')
Note

The resulting index from a set operation will be sorted in ascending order.

When performing Index.union() between indexes with different dtypes, the indexes must be cast to a common dtype. Typically, though not always, this is object dtype. The exception is when performing a union between integer and float data. In this case, the integer values are converted to float

In [331]: idx1 = pd.Index([0, 1, 2])

In [332]: idx2 = pd.Index([0.5, 1.5])

In [333]: idx1.union(idx2)
Out[333]: Float64Index([0.0, 0.5, 1.0, 1.5, 2.0], dtype='float64')
Missing values
Important

Even though Index can hold missing values (NaN), it should be avoided if you do not want any unexpected results. For example, some operations exclude missing values implicitly.

Index.fillna fills missing values with specified scalar value.

In [334]: idx1 = pd.Index([1, np.nan, 3, 4])

In [335]: idx1
Out[335]: Float64Index([1.0, nan, 3.0, 4.0], dtype='float64')

In [336]: idx1.fillna(2)
Out[336]: Float64Index([1.0, 2.0, 3.0, 4.0], dtype='float64')

In [337]: idx2 = pd.DatetimeIndex([pd.Timestamp('2011-01-01'),
   .....:                          pd.NaT,
   .....:                          pd.Timestamp('2011-01-03')])
   .....: 

In [338]: idx2
Out[338]: DatetimeIndex(['2011-01-01', 'NaT', '2011-01-03'], dtype='datetime64[ns]', freq=None)

In [339]: idx2.fillna(pd.Timestamp('2011-01-02'))
Out[339]: DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03'], dtype='datetime64[ns]', freq=None)
Set / reset index
Occasionally you will load or create a data set into a DataFrame and want to add an index after you’ve already done so. There are a couple of different ways.

Set an index
DataFrame has a set_index() method which takes a column name (for a regular Index) or a list of column names (for a MultiIndex). To create a new, re-indexed DataFrame:

In [340]: data
Out[340]: 
     a    b  c    d
0  bar  one  z  1.0
1  bar  two  y  2.0
2  foo  one  x  3.0
3  foo  two  w  4.0

In [341]: indexed1 = data.set_index('c')

In [342]: indexed1
Out[342]: 
     a    b    d
c               
z  bar  one  1.0
y  bar  two  2.0
x  foo  one  3.0
w  foo  two  4.0

In [343]: indexed2 = data.set_index(['a', 'b'])

In [344]: indexed2
Out[344]: 
         c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0
The append keyword option allow you to keep the existing index and append the given columns to a MultiIndex:

In [345]: frame = data.set_index('c', drop=False)

In [346]: frame = frame.set_index(['a', 'b'], append=True)

In [347]: frame
Out[347]: 
           c    d
c a   b          
z bar one  z  1.0
y bar two  y  2.0
x foo one  x  3.0
w foo two  w  4.0
Other options in set_index allow you not drop the index columns or to add the index in-place (without creating a new object):

In [348]: data.set_index('c', drop=False)
Out[348]: 
     a    b  c    d
c                  
z  bar  one  z  1.0
y  bar  two  y  2.0
x  foo  one  x  3.0
w  foo  two  w  4.0

In [349]: data.set_index(['a', 'b'], inplace=True)

In [350]: data
Out[350]: 
         c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0
Reset the index
As a convenience, there is a new function on DataFrame called reset_index() which transfers the index values into the DataFrame’s columns and sets a simple integer index. This is the inverse operation of set_index().

In [351]: data
Out[351]: 
         c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

In [352]: data.reset_index()
Out[352]: 
     a    b  c    d
0  bar  one  z  1.0
1  bar  two  y  2.0
2  foo  one  x  3.0
3  foo  two  w  4.0
The output is more similar to a SQL table or a record array. The names for the columns derived from the index are the ones stored in the names attribute.

You can use the level keyword to remove only a portion of the index:

In [353]: frame
Out[353]: 
           c    d
c a   b          
z bar one  z  1.0
y bar two  y  2.0
x foo one  x  3.0
w foo two  w  4.0

In [354]: frame.reset_index(level=1)
Out[354]: 
         a  c    d
c b               
z one  bar  z  1.0
y two  bar  y  2.0
x one  foo  x  3.0
w two  foo  w  4.0
reset_index takes an optional parameter drop which if true simply discards the index, instead of putting index values in the DataFrame’s columns.

Adding an ad hoc index
If you create an index yourself, you can just assign it to the index field:

data.index = index
Returning a view versus a copy
When setting values in a pandas object, care must be taken to avoid what is called chained indexing. Here is an example.

In [355]: dfmi = pd.DataFrame([list('abcd'),
   .....:                      list('efgh'),
   .....:                      list('ijkl'),
   .....:                      list('mnop')],
   .....:                     columns=pd.MultiIndex.from_product([['one', 'two'],
   .....:                                                         ['first', 'second']]))
   .....: 

In [356]: dfmi
Out[356]: 
    one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p
Compare these two access methods:

In [357]: dfmi['one']['second']
Out[357]: 
0    b
1    f
2    j
3    n
Name: second, dtype: object
In [358]: dfmi.loc[:, ('one', 'second')]
Out[358]: 
0    b
1    f
2    j
3    n
Name: (one, second), dtype: object
These both yield the same results, so which should you use? It is instructive to understand the order of operations on these and why method 2 (.loc) is much preferred over method 1 (chained []).

dfmi['one'] selects the first level of the columns and returns a DataFrame that is singly-indexed. Then another Python operation dfmi_with_one['second'] selects the series indexed by 'second'. This is indicated by the variable dfmi_with_one because pandas sees these operations as separate events. e.g. separate calls to __getitem__, so it has to treat them as linear operations, they happen one after another.

Contrast this to df.loc[:,('one','second')] which passes a nested tuple of (slice(None),('one','second')) to a single call to __getitem__. This allows pandas to deal with this as a single entity. Furthermore this order of operations can be significantly faster, and allows one to index both axes if so desired.

Why does assignment fail when using chained indexing?
The problem in the previous section is just a performance issue. What’s up with the SettingWithCopy warning? We don’t usually throw warnings around when you do something that might cost a few extra milliseconds!

But it turns out that assigning to the product of chained indexing has inherently unpredictable results. To see this, think about how the Python interpreter executes this code:

dfmi.loc[:, ('one', 'second')] = value
# becomes
dfmi.loc.__setitem__((slice(None), ('one', 'second')), value)
But this code is handled differently:

dfmi['one']['second'] = value
# becomes
dfmi.__getitem__('one').__setitem__('second', value)
See that __getitem__ in there? Outside of simple cases, it’s very hard to predict whether it will return a view or a copy (it depends on the memory layout of the array, about which pandas makes no guarantees), and therefore whether the __setitem__ will modify dfmi or a temporary object that gets thrown out immediately afterward. That’s what SettingWithCopy is warning you about!

Note

You may be wondering whether we should be concerned about the loc property in the first example. But dfmi.loc is guaranteed to be dfmi itself with modified indexing behavior, so dfmi.loc.__getitem__ / dfmi.loc.__setitem__ operate on dfmi directly. Of course, dfmi.loc.__getitem__(idx) may be a view or a copy of dfmi.

Sometimes a SettingWithCopy warning will arise at times when there’s no obvious chained indexing going on. These are the bugs that SettingWithCopy is designed to catch! pandas is probably trying to warn you that you’ve done this:

def do_something(df):
    foo = df[['bar', 'baz']]  # Is foo a view? A copy? Nobody knows!
    # ... many lines here ...
    # We don't know whether this will modify df or not!
    foo['quux'] = value
    return foo
Yikes!

Evaluation order matters
When you use chained indexing, the order and type of the indexing operation partially determine whether the result is a slice into the original object, or a copy of the slice.

pandas has the SettingWithCopyWarning because assigning to a copy of a slice is frequently not intentional, but a mistake caused by chained indexing returning a copy where a slice was expected.

If you would like pandas to be more or less trusting about assignment to a chained indexing expression, you can set the option mode.chained_assignment to one of these values:

'warn', the default, means a SettingWithCopyWarning is printed.

'raise' means pandas will raise a SettingWithCopyException you have to deal with.

None will suppress the warnings entirely.

In [359]: dfb = pd.DataFrame({'a': ['one', 'one', 'two',
   .....:                           'three', 'two', 'one', 'six'],
   .....:                     'c': np.arange(7)})
   .....: 

# This will show the SettingWithCopyWarning
# but the frame values will be set
In [360]: dfb['c'][dfb['a'].str.startswith('o')] = 42
This however is operating on a copy and will not work.

>>> pd.set_option('mode.chained_assignment','warn')
>>> dfb[dfb['a'].str.startswith('o')]['c'] = 42
Traceback (most recent call last)
     ...
SettingWithCopyWarning:
     A value is trying to be set on a copy of a slice from a DataFrame.
     Try using .loc[row_index,col_indexer] = value instead
A chained assignment can also crop up in setting in a mixed dtype frame.

Note

These setting rules apply to all of .loc/.iloc.

The following is the recommended access method using .loc for multiple items (using mask) and a single item using a fixed index:

In [361]: dfc = pd.DataFrame({'a': ['one', 'one', 'two',
   .....:                           'three', 'two', 'one', 'six'],
   .....:                     'c': np.arange(7)})
   .....: 

In [362]: dfd = dfc.copy()

# Setting multiple items using a mask
In [363]: mask = dfd['a'].str.startswith('o')

In [364]: dfd.loc[mask, 'c'] = 42

In [365]: dfd
Out[365]: 
       a   c
0    one  42
1    one  42
2    two   2
3  three   3
4    two   4
5    one  42
6    six   6

# Setting a single item
In [366]: dfd = dfc.copy()

In [367]: dfd.loc[2, 'a'] = 11

In [368]: dfd
Out[368]: 
       a  c
0    one  0
1    one  1
2     11  2
3  three  3
4    two  4
5    one  5
6    six  6
The following can work at times, but it is not guaranteed to, and therefore should be avoided:

In [369]: dfd = dfc.copy()

In [370]: dfd['a'][2] = 111

In [371]: dfd
Out[371]: 
       a  c
0    one  0
1    one  1
2    111  2
3  three  3
4    two  4
5    one  5
6    six  6
Last, the subsequent example will not work at all, and so should be avoided:

>>> pd.set_option('mode.chained_assignment','raise')
>>> dfd.loc[0]['a'] = 1111
Traceback (most recent call last)
     ...
SettingWithCopyException:
     A value is trying to be set on a copy of a slice from a DataFrame.
     Try using .loc[row_index,col_indexer] = value instead
Warning

The chained assignment warnings / exceptions are aiming to inform the user of a possibly invalid assignment. There may be false positives; situations where a chained assignment is inadvertently reported.